In [11]:
## Import Modules 
import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
from torchsummary import summary

In [12]:
## Model Configuration (Initialize hyperparameters)
batch_size = 64
learning_rate = 0.01
cross_entropy = nn.CrossEntropyLoss()

In [13]:
## DataLoader (Load the training set and validation set using Dataset and DataLoader)
transform = torchvision.transforms.ToTensor()
train_data = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
    'mnist_data', train=True, download=True, transform=transform
    ), batch_size=batch_size
)
val_data = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
    'mnist_data', train=False, download=True, transform=transform
    ), batch_size=batch_size
)

In [14]:
# Validation function (To check whether the model is learning properly we can use a validation set)
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        images = images.cuda()
        labels = labels.cuda()
        y_pred = model(images)
        value, pred = torch.max(y_pred, 1)
        total += y_pred.size(0)
        correct += torch.sum(pred == labels)
    return correct * 100 / total

In [15]:
## Training Function (Training the model)
def train(model,epochs=5) :
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)    
    for n in range(epochs)  :
        for i , (images , labels) in enumerate(train_data) :
            images = images.cuda()
            labels = labels.cuda()
            optimizer.zero_grad()
            prediction = model(images)
            loss = cross_entropy(prediction, labels)
            loss.backward()
            optimizer.step()
        accuracy = float(validate(model, val_data))
        print("Epoch:", n+1, "Loss: ", float(loss.data), "Accuracy:", accuracy)

In [16]:
## Model (A sample CNN is defined here for image classification)
class CNN(nn.Module) :
    def __init__(self):
        super(CNN,self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1,out_channels=8,kernel_size=3)
        self.conv_2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3)
        self.dense_1 = nn.Linear(in_features=9216,out_features=2048)
        self.dense_2 = nn.Linear(in_features=2048,out_features=10)

        self.tanh = nn.Tanh()
    def forward(self,x) :
        x = self.tanh(self.conv_1(x))
        x = self.tanh(self.conv_2(x))
        x = x.view(x.shape[0],-1)
        x = self.tanh(self.dense_1(x))
        x = self.dense_2(x)
        # output = self.tanh(x)
        output = F.log_softmax(x, dim=1)

        return output

In [17]:
# Model (Initialize the neural network)
model = CNN().cuda()

In [18]:
# Summary
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              Tanh-2            [-1, 8, 26, 26]               0
            Conv2d-3           [-1, 16, 24, 24]           1,168
              Tanh-4           [-1, 16, 24, 24]               0
            Linear-5                 [-1, 2048]      18,876,416
              Tanh-6                 [-1, 2048]               0
            Linear-7                   [-1, 10]          20,490
Total params: 18,898,154
Trainable params: 18,898,154
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.25
Params size (MB): 72.09
Estimated Total Size (MB): 72.35
----------------------------------------------------------------


In [19]:
# Train for 30 Epochs
train(model,epochs=30)

Epoch: 1 Loss:  0.5748899579048157 Accuracy: 75.75
Epoch: 2 Loss:  0.36637410521507263 Accuracy: 84.66999816894531
Epoch: 3 Loss:  0.08912859857082367 Accuracy: 90.72999572753906
Epoch: 4 Loss:  0.9771987199783325 Accuracy: 78.33000183105469
Epoch: 5 Loss:  0.2745267450809479 Accuracy: 81.8499984741211
Epoch: 6 Loss:  1.6649562120437622 Accuracy: 88.80999755859375
Epoch: 7 Loss:  0.5732375979423523 Accuracy: 86.0
Epoch: 8 Loss:  0.03406352922320366 Accuracy: 86.13999938964844
Epoch: 9 Loss:  0.17515763640403748 Accuracy: 88.6199951171875
Epoch: 10 Loss:  0.00011473985796328634 Accuracy: 84.79999542236328
Epoch: 11 Loss:  0.3487076759338379 Accuracy: 83.45999908447266
Epoch: 12 Loss:  1.0199308395385742 Accuracy: 83.91999816894531
Epoch: 13 Loss:  0.008138145320117474 Accuracy: 89.5999984741211
Epoch: 14 Loss:  0.8222111463546753 Accuracy: 92.43000030517578
Epoch: 15 Loss:  1.0973507165908813 Accuracy: 84.0999984741211
Epoch: 16 Loss:  2.3473243713378906 Accuracy: 88.06999969482422
Epoc

**Here, The model is reducing the loss function with varying validation accuracies.**